In [89]:
from datetime import datetime
import pandas as pd
import numpy as np
import math

## Загружаем данные, приведенные в порядок

In [9]:
parametrs1 = pd.read_excel('ВАЖНО база с нужными признаками Параметры МНЛЗ.xlsx')
parametrs = parametrs1.copy()

In [29]:
parametrs = parametrs1.copy()

In [30]:
len(parametrs)

22428

#### для удаления столбцов

In [31]:
def parametrs_drop(name_col): 
    parametrs.drop([name_col], axis='columns', inplace=True) 
    

#### Для отображения столбцов

In [42]:
def df_col(df):
    print('Всего столбцов: ',len(df.columns), '\n')
    for i in range(len(df.columns)):
        print(df.columns[i])
    

.

In [32]:
parametrs.head(2)

,Unnamed: 0,№ п/п,Паспорт разливки плавки,Дата,"Вес заготовок, т",Марка стали полученная,ГОСТ,Плавка в серии,"Вес стали расчет., т","Вес разлитой стали, т",...,"Ca, %","N, %","W, %","Bi, %","Pb, %","Sb, %","Mg, %","Zn, %","Сэ, %",Разность температур при замерах
0,0,1,12004615,2020-07-01,"168,96",Арм500,ГОСТ 34028-2016,5.0,"169,36",168.960,...,0.0008,0.0051,NaN,NaN,0.0032,NaN,0.0001,0.0036,0.425,3.0
1,1,2,12004616,2020-07-01,"165,888",Арм500,ГОСТ 34028-2016,6.0,"166,288",165.888,...,0.0010,0.0049,NaN,NaN,0.0034,NaN,0.0001,0.0029,0.422,0.0


In [33]:
parametrs_drop('Unnamed: 0')

In [35]:
parametrs.head(1)

,№ п/п,Паспорт разливки плавки,Дата,"Вес заготовок, т",Марка стали полученная,ГОСТ,Плавка в серии,"Вес стали расчет., т","Вес разлитой стали, т","Темп. стали в с/к, °C",...,"Ca, %","N, %","W, %","Bi, %","Pb, %","Sb, %","Mg, %","Zn, %","Сэ, %",Разность температур при замерах
0,1,12004615,2020-07-01,"168,96",Арм500,ГОСТ 34028-2016,5.0,"169,36",168.96,1565.0,...,0.0008,0.0051,NaN,NaN,0.0032,NaN,0.0001,0.0036,0.425,3.0


In [43]:
df_col(parametrs)

Всего столбцов:  48 

№ п/п
Паспорт разливки плавки
Дата
Вес заготовок, т
Марка стали полученная
ГОСТ
Плавка в серии
Вес стали расчет., т
Вес разлитой стали, т
Темп. стали в с/к, °C
Стойкость гильзы кр-ра 1, т
Частота качания, кол-во/мин
Ход кр-ра, мм
Скорость разливки, м/мин
Расход воды на кр-р, л/мин
Дельта температуры воды, °C
Расход воды ЗВО №1, л/мин
Расход воды ЗВО №2, л/мин
Расход воды ЗВО №3, л/мин
Брак всего, кг
Марка
Кол-во, т
Время пробы  МНЛЗ
C, %
Si, %
Mn,%
S, %
P, %
Cr, %
Ni, %
Cu, %
As, %
Mo, %
Nb, %
Sn, %
Ti, %
V, %
Al, %
Ca, %
N, %
W, %
Bi, %
Pb, %
Sb, %
Mg, %
Zn, %
Сэ, %
Разность температур при замерах


In [45]:
passports1 = pd.read_excel('паспорта 2020.xlsx')
passports = passports1.copy()
passports.head(1)

,id гильзы,Суммарная Стойкость,Ревизий
0,30014811,11111,1.0


In [50]:
passports_all1 = pd.read_excel('ВСЕ ДАННЫЕ паспорта в приведенном виде.xlsx')
passports_all = passports_all1.copy()
print(len(passports_all))
passports_all.head(1)

420


,№ гильзы,Дата установки,Дата вывода,Причина,"Стойкость, т",Зазор по шаблону,R1,r1,Левый,Правый,Замечания,№ кристаллизатора,№ ручья
0,30014811,2020-08-04 00:00:00,2020-08-06,ромб 18,11111.0,0.4,0.4,0.4,0.4,0.4,NaN,NaN,NaN


In [52]:
# именно эти значения стойкости мы будем искать в таблице с параметрами
passports_all['Стойкость, т']

0      11111.0
1       3984.0
2       2301.0
3          NaN
4       2506.0
        ...   
415        NaN
416        NaN
417        NaN
418        NaN
419        NaN
Name: Стойкость, т, Length: 420, dtype: float64

### Пишем алгоритм который подставит id гильзы в таблицу parametrs

In [46]:
parametrs['Стойкость гильзы кр-ра 1, т']

0        1122.0
1        1150.0
2        1174.0
3        1202.0
4        1226.0
          ...  
22423     854.0
22424     875.0
22425    1008.0
22426    1062.0
22427    1117.0
Name: Стойкость гильзы кр-ра 1, т, Length: 22428, dtype: float64

In [127]:
parametrs['key'] = 0
for i in range(1,len(parametrs['Стойкость гильзы кр-ра 1, т'])):
    try:
        if np.isnan(parametrs['Стойкость гильзы кр-ра 1, т'][i]):
            parametrs['Стойкость гильзы кр-ра 1, т'][i] = parametrs['Стойкость гильзы кр-ра 1, т'][i-1]
            print(parametrs['Стойкость гильзы кр-ра 1, т'][i])
        else:
            print(parametrs['Стойкость гильзы кр-ра 1, т'][i])
        if abs(parametrs['Стойкость гильзы кр-ра 1, т'][i] - parametrs['Стойкость гильзы кр-ра 1, т'][i-1])>400:

            print(parametrs['Стойкость гильзы кр-ра 1, т'][i-1],i-1)

            idx_pass = passports_all.index[passports_all['Стойкость, т'] == parametrs['Стойкость гильзы кр-ра 1, т'][i-1]][0]
            #passports_all['№ гильзы'][idx_pass]

            parametrs['key'][i-1] = passports_all['№ гильзы'][idx_pass]
            #break
    except:
        continue

1150.0
1174.0
1202.0
1226.0
1254.0
1279.0
1306.0
1334.0
1358.0
1386.0
1414.0
1438.0
1466.0
1493.0
1521.0
1546.0
1573.0
1601.0
1625.0
1653.0
1678.0
1705.0
1733.0
1760.0
1791.0
1819.0
1846.0
1874.0
1905.0
1932.0
1960.0
1987.0
2012.0
2018.0
2030.0
2052.0
2073.0
2092.0
2113.0
2135.0
31.0
2135.0 40
65.0
92.0
120.0
147.0
175.0
203.0
230.0
261.0
288.0
316.0
344.0
371.0
399.0
426.0
454.0
482.0
509.0
537.0
564.0
589.0
617.0
644.0
672.0
699.0
727.0
755.0
782.0
810.0
841.0
868.0
896.0
923.0
951.0
979.0
1009.0
1034.0
1061.0
1092.0
1120.0
1147.0
1175.0
1203.0
1230.0
1261.0
1288.0
1316.0
1344.0
1371.0
1399.0
1426.0
1454.0
1482.0
1509.0
1534.0
1561.0
1589.0
1620.0
1647.0
1675.0
1703.0
1733.0
1761.0
1792.0
1819.0
1850.0
1877.0
1908.0
1936.0
1966.0
1997.0
2028.0
2055.0
2086.0
2114.0
2141.0
2172.0
2200.0
2227.0
2255.0
2282.0
2310.0
2334.0
2365.0
2393.0
2420.0
2448.0
2472.0
2500.0
2525.0
2549.0
2577.0
2601.0
2629.0
2657.0
2684.0
2712.0
2739.0
2767.0
2795.0
2822.0
2850.0
2877.0
2905.0
2936.0
2963.0
2991.0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()



6405.0
6433.0
6463.0
6491.0
6522.0
6549.0
6577.0
6604.0
6632.0
6663.0
6690.0
6718.0
6745.0
6773.0
6797.0
6825.0
6853.0
6877.0
6905.0
6932.0
6957.0
6988.0
7018.0
7046.0
7074.0
7101.0
7129.0
7153.0
7178.0
7206.0
7233.0
7261.0
7288.0
7319.0
7347.0
7381.0
7408.0
7439.0
7466.0
7497.0
7528.0
7553.0
7583.0
7617.0
7651.0
7685.0
7715.0
7743.0
7771.0
7798.0
7826.0
7853.0
7884.0
7912.0
7942.0
7970.0
8001.0
8038.0
8071.0
8105.0
8139.0
8176.0
8204.0
8231.0
8259.0
8286.0
8314.0
8345.0
8372.0
8400.0
8427.0
8455.0
8483.0
8510.0
8538.0
8565.0
8593.0
8621.0
8648.0
8679.0
8707.0
8734.0
8762.0
8789.0
8817.0
8845.0
8875.0
8903.0
8930.0
8958.0
8986.0
9013.0
9041.0
9068.0
9099.0
9127.0
9154.0
9182.0
9210.0
9237.0
28.0
9237.0 1509
55.0
83.0
110.0
135.0
160.0
187.0
215.0
242.0
270.0
298.0
325.0
353.0
380.0
408.0
436.0
460.0
491.0
519.0
546.0
571.0
598.0
626.0
654.0
678.0
706.0
733.0
758.0
786.0
813.0
841.0
868.0
896.0
924.0
951.0
979.0
1006.0
1040.0
1071.0
1101.0
1132.0
1166.0
1200.0
1233.0
1261.0
1286.0
1313

1412.0 3610
5293.0
5493.0
5520.0
5547.0
5577.0
5880.0
6288.0
5880.0 3617
6343.0
6536.0
6589.0
6725.0
6780.0
6973.0
7028.0
7221.0
3440.0
7221.0 3626
3467.0
3799.0
3827.0
3769.0
3857.0
4250.0
4280.0
4307.0
1475.0
4307.0 3635
1693.0
1744.0
1877.0
1935.0
2137.0
2198.0
6066.0
2198.0 3642
6122.0
7277.0
6122.0 3644
7418.0
2303.0
7418.0 3646
2497.0
2524.0
2711.0
2739.0
2847.0
2902.0
4335.0
2902.0 3653
4632.0
4658.0
4708.0
4955.0
5015.0
5043.0
5070.0
5435.0
6261.0
5435.0 3662
6316.0
6509.0
6561.0
6752.0
6807.0
6946.0
7000.0
7193.0
5405.0
7193.0 3671
5378.0
5686.0
5713.0
5741.0
5992.0
3059.0
5992.0 3677
3086.0
3249.0
3385.0
3524.0
3579.0
3715.0
3884.0
3941.0
7250.0
3941.0 3686
2279.0
7250.0 3687
2334.0
2470.0
2548.0
2681.0
2875.0
2926.0
3032.0
4138.0
3032.0 3695
4195.0
4390.0
4444.0
4537.0
4569.0
4682.0
4763.0
4928.0
3219.0
4928.0 3704
3497.0
3552.0
3688.0
3742.0
3911.0
3969.0
4168.0
4222.0
4985.0
4222.0 3713
5097.0
5154.0
5324.0
5351.0
5463.0
5605.0
5768.0
5825.0
5964.0
5937.0
4363.0
5937.0 372

9553.0
9581.0
9608.0
9639.0
9664.0
9691.0
9691.0
9691.0
9694.0
9722.0
9749.0
9774.0
9799.0
9823.0
9848.0
9875.0
9900.0
9934.0
9961.0
9989.0
10014.0
10041.0
10068.0
10097.0
10128.0
10152.0
10187.0
10219.0
10248.0
10277.0
10308.0
10338.0
10367.0
10401.0
10428.0
10456.0
10484.0
10511.0
10539.0
10569.0
10600.0
10631.0
10662.0
10692.0
10723.0
10754.0
10781.0
10809.0
10840.0
10867.0
10898.0
10925.0
10953.0
10984.0
11011.0
11039.0
11067.0
11094.0
11125.0
11152.0
11177.0
11208.0
11235.0
11266.0
10018.0
11266.0 4788
10045.0
10070.0
100097.0
10070.0 4791
10125.0
100097.0 4792
10150.0
10174.0
10202.0
10229.0
10257.0
10282.0
10309.0
10337.0
10364.0
10392.0
10417.0
10444.0
10469.0
10493.0
10518.0
10543.0
10567.0
10592.0
10616.0
10644.0
10672.0
10699.0
10721.0
10745.0
10770.0
10795.0
10819.0
10841.0
10865.0
10890.0
10911.0
10936.0
10961.0
10985.0
10985.0
11034.0
11059.0
11084.0
11111.0
6423.0
11111.0 4832
6450.0
6478.0
6505.0
6533.0
6561.0
6588.0
6616.0
6643.0
6671.0
6699.0
6726.0
6754.0
6781.0
6809

...

4057.0
4247.0
4305.0
4450.0
4519.0
3435.0
4519.0 14244
3574.0
3622.0
3834.0
3891.0
4029.0
4084.0
4138.0
4277.0
4334.0
4655.0
4709.0
3427.0
4709.0 14256
3454.0
3482.0
3509.0
3866.0
1398.0
3866.0 14261
1425.0
1453.0
1891.0
1453.0 14264
3184.0
1891.0 14265
3236.0
3265.0
3293.0
3646.0
3673.0
3700.0
3730.0
1643.0
3730.0 14273
1918.0
1948.0
1975.0
2356.0
2383.0
2411.0
2425.0
2568.0
2589.0
1682.0
2589.0 14283
1722.0
1749.0
2117.0
2145.0
2169.0
2196.0
2196.0
2196.0
2613.0
2196.0 14292
2637.0
2661.0
3944.0
2661.0 14295
3969.0
3996.0
4032.0
4416.0
4446.0
4446.0
2453.0
4446.0 14302
2803.0
2834.0
2861.0
2891.0
4171.0
2891.0 14307
4198.0
4227.0
4473.0
4500.0
4885.0
4912.0
4935.0
4964.0
5359.0
5389.0
5418.0
4255.0
5418.0 14319
4282.0
4639.0
4668.0
4696.0
4723.0
5124.0
4723.0 14325
5155.0
5183.0
5212.0
5617.0
5212.0 14329
5645.0
5674.0
5704.0
6016.0
6044.0
6071.0
6096.0
5445.0
6096.0 14337
5851.0
5445.0 14338
5878.0
5888.0
5888.0
6264.0
6292.0
6319.0
6346.0
6485.0
6512.0
6539.0
6567.0
6959.0
6987.0
7

11900.0
11925.0
11949.0
11977.0
12002.0
12026.0
12051.0
12075.0
12103.0
12128.0
12155.0
12180.0
12204.0
12229.0
12254.0
12278.0
12303.0
12330.0
12358.0
12383.0
12407.0
12432.0
12456.0
12481.0
12503.0
12527.0
12552.0
12576.0
12601.0
12610.0
16635.0
12610.0 16456
12656.0
16635.0 16457
12690.0
12718.0
12745.0
28.0
12745.0 16461
55.0
83.0
110.0
138.0
166.0
193.0
221.0
245.0
273.0
301.0
328.0
356.0
383.0
411.0
439.0
463.0
488.0
509.0
537.0
565.0
592.0
620.0
647.0
672.0
697.0
724.0
749.0
776.0
804.0
832.0
859.0
887.0
914.0
942.0
970.0
997.0
1031.0
1062.0
1095.0
1126.0
1157.0
1191.0
1224.0
1252.0
1280.0
1307.0
1335.0
1362.0
1390.0
1418.0
1446.0
1470.0
1495.0
1522.0
1550.0
1578.0
1605.0
1633.0
1657.0
1657.0
1710.0
1737.0
1765.0
1792.0
1817.0
1845.0
1872.0
1897.0
1924.0
1952.0
1980.0
2007.0
2032.0
2059.0
2087.0
2118.0
2145.0
2173.0
2201.0
2228.0
2256.0
2283.0
2311.0
2339.0
2366.0
2394.0
2421.0
2446.0
2474.0
2501.0
2529.0
2556.0
2581.0
2605.0
2630.0
2655.0
2682.0
2710.0
2734.0
2762.0
2790.0
2817

7987.0
8015.0
8046.0
8073.0
8101.0
8135.0
8165.0
8193.0
8221.0
8248.0
8276.0
8300.0
8328.0
8356.0
8380.0
8408.0
8435.0
8463.0
8491.0
8518.0
8543.0
8567.0
8592.0
8620.0
8645.0
8654.0
8654.0
8654.0
8672.0
8694.0
8715.0
8737.0
8755.0
8777.0
8798.0
8820.0
8841.0
8863.0
8884.0
8906.0
8927.0
8952.0
8976.0
9001.0
9026.0
9050.0
9075.0
9099.0
9124.0
9152.0
9176.0
9204.0
9231.0
9259.0
9287.0
9315.0
9342.0
9367.0
9394.0
9419.0
9444.0
9468.0
9493.0
9499.0
7778.0
9499.0 17762
7805.0
7830.0
7854.0
7879.0
7907.0
7931.0
7956.0
7980.0
8005.0
8033.0
8057.0
8085.0
8109.0
8137.0
8162.0
8189.0
8217.0
8244.0
8272.0
8300.0
8324.0
8324.0
8349.0
8379.0
8410.0
8441.0
8475.0
8505.0
8533.0
8560.0
8588.0
8616.0
8643.0
8677.0
8708.0
8742.0
8773.0
8806.0
8837.0
8868.0
8899.0
8930.0
8960.0
8994.0
9025.0
9056.0
9087.0
9111.0
9139.0
9167.0
9198.0
9226.0
9253.0
9281.0
9281.0
9337.0
9364.0
9392.0
9423.0
9451.0
9479.0
9509.0
9540.0
9568.0
9596.0
9627.0
9651.0
9679.0
9707.0
2658.0
9707.0 17832
2822.0
2849.0
2906.0
3048.0
3

12295.0
12323.0
12351.0
12378.0
12406.0
12430.0
12455.0
12482.0
12510.0
12537.0
12562.0
12590.0
12614.0
12642.0
12666.0
12691.0
12719.0
12743.0
12771.0
12798.0
12826.0
12857.0
12884.0
12912.0
12940.0
12967.0
12992.0
13019.0
13044.0
13068.0
13096.0
13120.0
13148.0
13173.0
13200.0
13225.0
13249.0
13277.0
13302.0
13326.0
13354.0
13381.0
13406.0
13434.0
13461.0
13486.0
13513.0
13538.0
13566.0
13590.0
13618.0
13642.0
13670.0
13695.0
13722.0
13750.0
8708.0
13750.0 19933
8735.0
8763.0
8790.0
8815.0
8846.0
8873.0
8901.0
8928.0
8953.0
8981.0
9008.0
9036.0
9060.0
9088.0
9116.0
9143.0
9171.0
9195.0
9226.0
9251.0
9278.0
9306.0
9334.0
9361.0
9361.0
9419.0
9444.0
9475.0
9502.0
9527.0
9555.0
9582.0
9607.0
9634.0
9662.0
9690.0
9714.0
9742.0
9769.0
9797.0
9828.0
9856.0
9880.0
9908.0
9935.0
9960.0
9988.0
10015.0
10040.0
10067.0
10095.0
10123.0
10150.0
10175.0
10202.0
10230.0
10258.0
10285.0
10310.0
10334.0
10362.0
10387.0
10414.0
10442.0
10466.0
10494.0
10522.0
10546.0
10574.0
10601.0
10629.0
10654.0
10

3833.0
3858.0
3882.0
3910.0
3935.0
3959.0
3987.0
4011.0
4039.0
4064.0
4091.0
4116.0
4143.0
4171.0
4196.0
4223.0
4251.0
4275.0
4306.0
4334.0
4361.0
4389.0
4423.0
3340.0
4423.0 21231
3367.0
3392.0
3423.0
3450.0
3478.0
3508.0
3536.0
3564.0
3588.0
3616.0
3643.0
3674.0
3699.0
3726.0
3754.0
3782.0
3809.0
3837.0
3864.0
3895.0
3923.0
3950.0
3978.0
4005.0
4033.0
4061.0
4088.0
4116.0
4147.0
4171.0
4199.0
4226.0
4254.0
4282.0
4309.0
4337.0
4364.0
4392.0
4420.0
4447.0
4475.0
4502.0
4533.0
4561.0
4588.0
4616.0
4643.0
4671.0
4699.0
4724.0
4748.0
4776.0
4800.0
4828.0
4853.0
4880.0
4905.0
4932.0
4960.0
4988.0
5012.0
5040.0
5067.0
5095.0
5123.0
5150.0
5178.0
5205.0
5233.0
5261.0
5285.0
5313.0
5340.0
5368.0
5393.0
5420.0
5448.0
5479.0
5506.0
5534.0
5564.0
5595.0
5623.0
5653.0
5681.0
5709.0
5739.0
5767.0
5798.0
5831.0
5865.0
6180.0
6204.0
6232.0
6256.0
6281.0
6306.0
6339.0
6370.0
6398.0
6428.0
6456.0
6487.0
6521.0
6548.0
6582.0
6616.0
6650.0
6683.0
6711.0
6742.0
6769.0
6800.0
6828.0
6858.0
6886.0
6917.0


In [128]:
len(parametrs[(parametrs['key']!=0)])
# 80 без abs когда -400
# if parametrs['Стойкость гильзы кр-ра 1, т'][i] - parametrs['Стойкость гильзы кр-ра 1, т'][i-1]<-400 :

115

In [135]:
parametrs.to_excel('оставить значения1.xlsx')

In [136]:
parametrs = pd.read_excel('оставить значения.xlsx')
parametrs.head()

,key,"Вес заготовок, т",Плавка в серии,"Вес стали расчет., т","Вес разлитой стали, т","Темп. стали в с/к, °C","Частота качания, кол-во/мин","Ход кр-ра, мм","Скорость разливки, м/мин","Расход воды на кр-р, л/мин",...,"As, %","Mo, %","Nb, %","Ti, %","V, %","Al, %","Ca, %","N, %",Разность температур при замерах,Осталось тонн
0,30014817,"165,768",40,"174,792",174.392,1567,200,7,2,2150,...,0.0011,0.0029,0.0003,0.0007,0.0018,0.0028,0.0010,0.0053,1,40297
1,30014803,"159,642",23,"167,993",167.593,1570,200,13,2,2155,...,0.0011,0.0023,0.0002,0.0006,0.0013,0.0026,0.0010,0.0081,0,25839
2,30014803,"162,714",58,"170,472",170.072,1569,200,13,2,2155,...,0.0010,0.0025,0.0003,0.0007,0.0013,0.0028,0.0004,0.0085,-2,24222
3,30014817,"159,655",22,"168,366",167.966,1562,200,7,2,2155,...,0.0012,0.0024,0.0006,0.0011,0.0035,0.0031,0.0007,0.0069,5,33313
4,30014803,"156,488",22,"163,992",163.592,1569,200,7,2,2155,...,0.0010,0.0028,0.0005,0.0011,0.0031,0.0023,0.0007,0.0077,2,21515


In [137]:
len(parametrs)

109

In [138]:
df_col(parametrs)

Всего столбцов:  33 

key
Вес заготовок, т
Плавка в серии
Вес стали расчет., т
Вес разлитой стали, т
Темп. стали в с/к, °C
Частота качания, кол-во/мин
Ход кр-ра, мм
Скорость разливки, м/мин
Расход воды на кр-р, л/мин
Дельта температуры воды, °C
Расход воды ЗВО №1, л/мин
Расход воды ЗВО №2, л/мин
Расход воды ЗВО №3, л/мин
Кол-во, т
C, %
Si, %
Mn,%
S, %
P, %
Cr, %
Ni, %
Cu, %
As, %
Mo, %
Nb, %
Ti, %
V, %
Al, %
Ca, %
N, %
Разность температур при замерах
Осталось тонн


### Предсказывать будем Осталось тонн

In [139]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

In [141]:
!pip install xgboost

In [175]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import model_selection, preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [184]:
df = pd.read_excel('оставить значения трэйн.xlsx')
test = pd.read_excel('оставить значения тест.xlsx')

In [185]:
# удаляем строки с NA значениями
df = df.dropna()
test = test.dropna()

In [186]:
X_train = pd.get_dummies(df).drop('Осталось тонн', axis=1)
y_train = df['Осталось тонн']

In [187]:
X_test = pd.get_dummies(test).drop('Осталось тонн', axis=1)
y_test = test['Осталось тонн']

In [188]:
print (len(X_train.columns))
print (len(X_test.columns))
# Приводим множество названий колонок к типу set, находим разность двух множеств.
print(set(X_train.columns) - set(X_test.columns))
print(set(X_test.columns) - set(X_train.columns))

157
83
{'key_30014144', 'Вес заготовок, т_150,375', 'key_30014811', 'Вес заготовок, т_168,294', 'key_30014824', 'Вес стали расчет., т_171,579', 'Вес заготовок, т_162,714', 'Вес заготовок, т_165,222', 'key_30014810', 'key_30014825', 'Вес заготовок, т_162,534', 'key_30014134', 'key_30014804', 'key_30014806', 'Вес стали расчет., т_177,167', 'Вес заготовок, т_159,527', 'key_30014803', 'Вес заготовок, т_159,642', 'key_30014817', 'key_30014818', 'Вес стали расчет., т_171,435', 'Вес заготовок, т_162,561', 'Вес заготовок, т_162,381', 'key_30013867', 'key_30014141', 'Вес заготовок, т_159,103', 'Вес заготовок, т_159,744', 'Вес заготовок, т_162,611', 'Вес стали расчет., т_174,06', 'key_30013873', 'Вес стали расчет., т_172,392', 'Вес заготовок, т_165,312', 'key_30013876', 'Вес стали расчет., т_157,417', 'key_30014808', 'Вес стали расчет., т_165,712', 'Вес стали расчет., т_172,553', 'Вес стали расчет., т_172,22', 'key_30014814', 'Вес стали расчет., т_160,672', 'Вес стали расчет., т_175,585', 'Вес з

In [189]:
columns = set(X_train.columns) | set(X_test.columns)
X_train = X_train.reindex(columns=columns).fillna(0)
X_test = X_test.reindex(columns=columns).fillna(0)

In [190]:
# Команда all проверяет, все ли значения из входного списка равны True
all(X_train.columns == X_test.columns)

True

In [263]:
for i in range(1,20,1):  
    model = xgb.XGBRegressor(seed=i,
                          n_estimators=1000,
                          max_depth=3,
                          learning_rate=0.34)
    model.fit(X_train, y_train)

    y_pred_train = model.predict(X_train)
    print (mean_squared_error(y_train, y_pred_train))

    y_pred = model.predict(X_test)
    print (mean_squared_error(y_test, y_pred),i/100,'\n')
#2968188.1261132173 0.29 
    

2.8942416353923518e-05
2968188.1261132173 0.01 

2.8942416353923518e-05
2968188.1261132173 0.02 

2.8942416353923518e-05
2968188.1261132173 0.03 

2.8942416353923518e-05
2968188.1261132173 0.04 

2.8942416353923518e-05
2968188.1261132173 0.05 

2.8942416353923518e-05
2968188.1261132173 0.06 

2.8942416353923518e-05
2968188.1261132173 0.07 

2.8942416353923518e-05
2968188.1261132173 0.08 

2.8942416353923518e-05
2968188.1261132173 0.09 

2.8942416353923518e-05
2968188.1261132173 0.1 

2.8942416353923518e-05
2968188.1261132173 0.11 

2.8942416353923518e-05
2968188.1261132173 0.12 

2.8942416353923518e-05
2968188.1261132173 0.13 

2.8942416353923518e-05
2968188.1261132173 0.14 

2.8942416353923518e-05
2968188.1261132173 0.15 

2.8942416353923518e-05
2968188.1261132173 0.16 

2.8942416353923518e-05
2968188.1261132173 0.17 

2.8942416353923518e-05
2968188.1261132173 0.18 

2.8942416353923518e-05
2968188.1261132173 0.19 



In [195]:
from sklearn.model_selection import GridSearchCV

In [227]:
grid_param = {  
    'n_estimators': [[i for i in range(0,200,40)]],
    'max_depth': [[i for i in range(2,6,1)]],
    'learning_rate': [[i/100 for i in range(5,55,10)]]
}

In [228]:
xgb_grid = GridSearchCV(model,
                        grid_param,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train, y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   2 out of   2 | elapsed:    0.0s finished


TypeError: 'list' object cannot be interpreted as an integer

In [256]:
for i in range(1,50,1):
    print(i/100)

0.01
0.02
0.03
0.04
0.05
0.06
0.07
0.08
0.09
0.1
0.11
0.12
0.13
0.14
0.15
0.16
0.17
0.18
0.19
0.2
0.21
0.22
0.23
0.24
0.25
0.26
0.27
0.28
0.29
0.3
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
0.41
0.42
0.43
0.44
0.45
0.46
0.47
0.48
0.49


In [218]:
GridSearchCV(cv=5, error_score='raise',
       estimator=xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=-1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1234,
       silent=True, subsample=0.8),
       iid=True, n_jobs=-1,
       fit_params={},
       param_grid = {  'n_estimators': [[i for i in range(0,200,40)]],
                     'max_depth': [[i for i in range(2,6,1)]],
                     'learning_rate': [[i/100 for i in range(5,55,10)]]},
       pre_dispatch='2*n_jobs', refit=True, scoring='neg_root_mean_squared_error', verbose=0)

TypeError: __init__() got an unexpected keyword argument 'fit_params'

In [216]:
zzz.fit(X_train, y_train)

AttributeError: 'dict' object has no attribute 'fit'

In [207]:
grid_param

{'n_estimators': [[0, 40, 80, 120, 160]],
 'max_depth': [[2, 3, 4, 5]],
 'learning_rate': [[0.05, 0.15, 0.25, 0.35, 0.45]]}

In [203]:
a = [i for i in range(0.05,55,0.1)]

TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
randra

In [172]:
id_vals = test.ID.values

clean_test = test.copy()
for f in clean_test.columns:
    if clean_test[f].dtype == 'object':
        label = preprocessing.LabelEncoder()
        label.fit(list(clean_test[f].values))
        clean_test[f] = label.transform(list(clean_test[f].values))
clean_test.fillna((-999), inplace=True)
test1 = clean_test.drop(['ID'],axis=1)
X_test = test1.values

In [176]:
model = xgb.XGBRegressor(seed=42,
                      n_estimators=100,
                      max_depth=6,
                      learning_rate=0.3)
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
print (mean_squared_error(y_train, y_pred_train))

y_pred = model.predict(X_test)
print (mean_squared_error(y_test, y_pred))

0.004415530983994647


ValueError: feature_names mismatch: ['Вес заготовок, т_161,28', 'key_30014144', 'Частота качания, кол-во/мин', 'Вес заготовок, т_150,375', 'key_30014811', 'Вес стали расчет., т_168,68', 'Вес заготовок, т_168,294', 'Вес стали расчет., т_163,992', 'key_30014824', 'Плавка в серии', 'N, %', 'Темп. стали в с/к, °C', 'Вес стали расчет., т_171,579', 'Вес заготовок, т_161,677', 'Вес заготовок, т_162,714', 'Вес заготовок, т_162,33', 'S, %', 'Ход кр-ра, мм', 'Nb, %', 'Вес заготовок, т_165,222', 'Расход воды на кр-р, л/мин', 'Марка стали полученная_Арм500', 'key_30014810', 'key_30014825', 'Вес заготовок, т_162,534', 'key_30014134', 'Вес заготовок, т_159,706', 'key_30014804', 'Si, %', 'key_30014806', 'ГОСТ_ГОСТ 10702-2016', 'Вес заготовок, т_153,151', 'Вес заготовок, т_159,527', 'Вес стали расчет., т_177,167', 'key_30014803', 'Mo, %', 'Вес заготовок, т_159,642', 'Марка стали полученная_20', 'Вес заготовок, т_160,272', 'Вес стали расчет., т_163,696', 'Вес заготовок, т_172,368', 'key_30014817', 'key

In [168]:
#xgb.XGBRegressor()

In [171]:
df

,ID,"Вес заготовок, т",Марка стали полученная,ГОСТ,Плавка в серии,"Вес стали расчет., т","Вес разлитой стали, т","Темп. стали в с/к, °C","Стойкость гильзы кр-ра 1, т","Частота качания, кол-во/мин",...,"Nb, %","Ti, %","V, %","Al, %","Ca, %","N, %",Разность температур при замерах,key,Стойкость всего,Осталось тонн
1,254,"159,642",Арм500,ГОСТ 34028-2016,23,"167,993",167.593,1570,4657,200,...,0.0002,0.0006,0.0013,0.0026,0.0010,0.0081,0,30014803,30496,25839
2,518,"162,714",Арм500,ГОСТ 34028-2016,58,"170,472",170.072,1569,6274,200,...,0.0003,0.0007,0.0013,0.0028,0.0004,0.0085,-2,30014803,30496,24222
3,608,"159,655",Арм500,ГОСТ 34028-2016,22,"168,366",167.966,1562,10364,200,...,0.0006,0.0011,0.0035,0.0031,0.0007,0.0069,5,30014817,43677,33313
4,707,"156,488",Арм500,ГОСТ 34028-2016,22,"163,992",163.592,1569,8981,200,...,0.0005,0.0011,0.0031,0.0023,0.0007,0.0077,2,30014803,30496,21515
5,835,"159,527",Арм500,ГОСТ 34028-2016,26,"166,438",166.038,1562,10316,200,...,0.0005,0.0012,0.0038,0.0024,0.0008,0.0076,-3,30014817,43677,33361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,16461,"168,294",Арм500,ГОСТ 34028-2016,59,"177,564",177.164,1566,12745,200,...,0.0005,0.0012,0.0030,0.0029,0.0011,0.0084,0,30014824,42235,29490
84,16680,"162,047",Арм500,ГОСТ 34028-2016,16,"172,803",172.403,1559,4986,200,...,0.0006,0.0015,0.0034,0.0038,0.0011,0.0062,1,30014818,7492,2506
85,17362,"156,506",Арм500,ГОСТ 34028-2016,33,"165,65",165.250,1574,3372,180,...,0.0003,0.0010,0.0023,0.0032,0.0018,0.0053,0,30014807,33397,30025
86,17982,"168,664",Арм500,ГОСТ 34028-2016,20,"174,364",173.964,1572,4422,180,...,0.0005,0.0008,0.0025,0.0021,0.0010,0.0063,3,30013363,8988,4566


In [152]:
train_y = clean_df['Осталось тонн'].values
train_x = clean_df.drop(["Осталось тонн"],axis=1)
train_x = train_x.drop(["ID"],axis=1)
train_x = train_x.values

In [151]:
clean_df = train_set.copy()

for f in clean_df.columns:
    if clean_df[f].dtype == 'object':
        label = preprocessing.LabelEncoder()
        label.fit(list(clean_df[f].values))
        clean_df[f] = label.transform(list(clean_df[f].values))
clean_df.head()

,ID,"Вес заготовок, т",Марка стали полученная,ГОСТ,Плавка в серии,"Вес стали расчет., т","Вес разлитой стали, т","Темп. стали в с/к, °C","Стойкость гильзы кр-ра 1, т","Частота качания, кол-во/мин",...,"Nb, %","Ti, %","V, %","Al, %","Ca, %","N, %",Разность температур при замерах,key,Стойкость всего,Осталось тонн
0,161,34,4,3,40,40,174.392,1567,3380,200,...,0.0003,0.0007,0.0018,0.0028,0.0010,0.0053,1,23,43677,40297
1,254,9,4,3,23,16,167.593,1570,4657,200,...,0.0002,0.0006,0.0013,0.0026,0.0010,0.0081,0,12,30496,25839
2,518,25,4,3,58,21,170.072,1569,6274,200,...,0.0003,0.0007,0.0013,0.0028,0.0004,0.0085,-2,12,30496,24222
3,608,10,4,3,22,17,167.966,1562,10364,200,...,0.0006,0.0011,0.0035,0.0031,0.0007,0.0069,5,23,43677,33313
4,707,2,4,3,22,8,163.592,1569,8981,200,...,0.0005,0.0011,0.0031,0.0023,0.0007,0.0077,2,12,30496,21515


In [153]:
#Fitting XGB regressor 
model = xgb.XGBRegressor()
model.fit(train_x,train_y)
print (model)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)


In [155]:
#Transforming the testset
id_vals = test_set.ID.values

clean_test = test_set.copy()
for f in clean_test.columns:
    if clean_test[f].dtype == 'object':
        label = preprocessing.LabelEncoder()
        label.fit(list(clean_test[f].values))
        clean_test[f] = label.transform(list(clean_test[f].values))
clean_test.fillna((-999), inplace=True)
test = clean_test.drop(['ID'],axis=1)
x_test = test.values

#testset is ready


In [156]:
#Predict 
output = model.predict(data=x_test)
final_df = pd.DataFrame()
final_df["ID"] = id_vals
final_df["Prediction"] = output
#final_df.to_csv("Output_1.csv")
final_df.head()

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36'] ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37']
training data did not have the following fields: f37

получилось мало данных попробуем усовершенствовать фильтр чтобы он сверялся еще и с датой вывода, но добавим погрешность к сравнению величин стойкости +-10(например) и посмотрим на количество заполненых key тогда еще раз

In [125]:
parametrs['key2'] = 0
l = []
for i in range(1,len(parametrs['Стойкость гильзы кр-ра 1, т'])):
    try:
        if np.isnan(parametrs['Стойкость гильзы кр-ра 1, т'][i]):
            parametrs['Стойкость гильзы кр-ра 1, т'][i] = parametrs['Стойкость гильзы кр-ра 1, т'][i-1]
            print(parametrs['Стойкость гильзы кр-ра 1, т'][i])
        else:
            pass#print(parametrs['Стойкость гильзы кр-ра 1, т'][i])
        if abs(parametrs['Стойкость гильзы кр-ра 1, т'][i] - parametrs['Стойкость гильзы кр-ра 1, т'][i-1])>400:

            print(parametrs['Стойкость гильзы кр-ра 1, т'][i-1],i-1)
            
            for j in range(1,len(passports_all['Стойкость, т'])):
                
                idx_pass = passports_all.index[passports_all['Стойкость, т'] > parametrs['Стойкость гильзы кр-ра 1, т'][i-1]][0]-20
                l.append(passports_all['Стойкость, т'][idx_pass])
            l.sort()
            idx_pass2 = l[0] 
            #passports_all['№ гильзы'][idx_pass]
            print(idx_pass2)
            if  passports_all['Дата вывода'][idx_pass2] == parametrs['Дата'][i-1]: 
                
                print('+')
                parametrs['key2'][i-1] = passports_all['№ гильзы'][idx_pass2]
            
            #break
    except:
        continue
#idx_pass = passports_all.index[passports_all['Стойкость, т'] == parametrs['Стойкость гильзы кр-ра 1, т'][i-1]][0]
#Дата вывода
#and parametrs['Стойкость гильзы кр-ра 1, т'][i-1]-1000 >=passports_all['Стойкость, т']][0]
#and passports_all['Дата вывода'][passports_all.index[passports_all['Стойкость, т'] >= parametrs['Стойкость гильзы кр-ра 1, т'][i-1]-1000 and parametrs['Стойкость гильзы кр-ра 1, т'][i-1]+1000 >=passports_all['Стойкость, т']][0]]==parametrs['Дата'][i-1]

2135.0 40
3380.0 161
4657.0 254
7884.0 417
5850.0 460
6274.0 518
10364.0 608
8981.0 707
9377.0 762
10316.0 835
801.0 865
286.0 866
1602.0 897
9785.0 928
4122.0 932
10703.0 1023
17731.0 1024
11409.0 1050
nan
2770.0 1156
12242.0 1193
nan
4581.0 1285
9237.0 1509
2492.0 1600
5616.0 1677
4832.0 1759
7238.0 1760
6311.0 1814
6759.0 1923
12064.0 1952
nan
10877.0 2102
8703.0 2217
15838.0 2333
nan
3642.0 2464
4234.0 2541
5732.0 2633
6190.0 2770
13582.0 2880
nan
5310.0 2888
2375.0 2898
5283.0 2905
1607.0 2907
3431.0 2916
4431.0 2943
6539.0 2970
6510.0 2988
4138.0 2997
4757.0 3010
5364.0 3024
4790.0 3030
5337.0 3047
1502.0 3050
1922.0 3052
5203.0 3059
2916.0 3069
2236.0 3076
3784.0 3087
4748.0 3105
4055.0 3111
5023.0 3119
5510.0 3123
5260.0 3128
6424.0 3140
3445.0 3141
6594.0 3148
4394.0 3159
3755.0 3163
4476.0 3171
4694.0 3180
6212.0 3204
5473.0 3213
7479.0 3222
6438.0 3231
8408.0 3240
7715.0 3249
8463.0 3251
4579.0 3258
6294.0 3267
8596.0 3276
5070.0 3281
7772.0 3285
7156.0 3294
8677.0 3297
4824

12576.0 15714
nan
2891.0 15824
14839.0 15857
nan
18863.0 15858
15600.0 15884
nan
17266.0 15904
3594.0 15913
9377.0 15914
4422.0 15943
10181.0 15944
6096.0 16002
2013.0 16075
8683.0 16167
4912.0 16274
4592.0 16297
6717.0 16298
5178.0 16319
6617.0 16372
10424.0 16402
12610.0 16456
nan
16635.0 16457
nan
12745.0 16461
nan
4006.0 16601
4986.0 16680
3353.0 16803
6731.0 16904
12506.0 17054
nan
10398.0 17156
8303.0 17158
10786.0 17169
10107.0 17245
3372.0 17362
10423.0 17416
4302.0 17451
3330.0 17452
5332.0 17493
9364.0 17585
9071.0 17690
9499.0 17762
9707.0 17832
3355.0 17840
3328.0 17857
1613.0 17859
3592.0 17868
4583.0 17895
6656.0 17922
6629.0 17940
3959.0 17949
4286.0 17962
7672.0 17971
571.0 17977
4422.0 17982
7645.0 17992
190.0 17994
3382.0 17999
1810.0 18003
3248.0 18011
2780.0 18023
2112.0 18028
2865.0 18033
3943.0 18039
3085.0 18045
4888.0 18057
5163.0 18071
5652.0 18075
5404.0 18080
6547.0 18092
3324.0 18093
6708.0 18100
4129.0 18111
4207.0 18123
7265.0 18129
4413.0 18132
7699.0 181

In [98]:
parametrs['Стойкость гильзы кр-ра 1, т'][i-1]

1062.0

In [99]:
passports_all['Стойкость, т'][idx_pass]

2256.0

In [119]:
len(parametrs[(parametrs['key2']!=0)])

0

In [107]:
parametrs['key2']

0        0
1        0
2        0
3        0
4        0
        ..
22423    0
22424    0
22425    0
22426    0
22427    0
Name: key2, Length: 22428, dtype: int64

In [73]:
passports_all['Дата вывода'][0]

Timestamp('2020-08-06 00:00:00')

In [74]:
parametrs['Дата'][0]

Timestamp('2020-07-01 00:00:00')

In [ ]:
idx_pass = passports_all.index[(parametrs['Стойкость гильзы кр-ра 1, т'][i-1]-20) < passports_all['Стойкость, т'] < (parametrs['Стойкость гильзы кр-ра 1, т'][i-1]+20)][0]

In [88]:
parametrs['Стойкость гильзы кр-ра 1, т'][40]#из таблицы параметров значение перепада
#parametrs['key'][40]

2135.0

In [56]:
passports_all.index[passports_all['Стойкость, т'] == 2135.0][0]

26

In [59]:
passports_all[26:27]

,№ гильзы,Дата установки,Дата вывода,Причина,"Стойкость, т",Зазор по шаблону,R1,r1,Левый,Правый,Замечания,№ кристаллизатора,№ ручья
26,30014802,2020-06-29 00:00:00,2020-07-02,продольная угловая трещина,2135.0,0.4,0.4,0.4,0.4,0.4,NaN,NaN,NaN


In [60]:
passports_all['№ гильзы'][26]

30014802

In [ ]:
for i in range(1,len(par_mnlz['Стойкость гильзы кр-ра 3, т'])):
    
    if np.isnan(par_mnlz['Стойкость гильзы кр-ра 3, т'][i]):
        par_mnlz['Стойкость гильзы кр-ра 3, т'][i] = par_mnlz['Стойкость гильзы кр-ра 3, т'][i-1]
        print(par_mnlz['Стойкость гильзы кр-ра 3, т'][i])
    else:
        print(par_mnlz['Стойкость гильзы кр-ра 3, т'][i])
    if par_mnlz['Стойкость гильзы кр-ра 3, т'][i] - par_mnlz['Стойкость гильзы кр-ра 3, т'][i-1]<-400:

        print(par_mnlz['Стойкость гильзы кр-ра 3, т'][i-1],i-1)
        break

In [111]:
a = [1,2,56,3,8,0,76,5,99]

In [115]:
a.sort()
a[0]

0

In [116]:
a

[0, 1, 2, 3, 5, 8, 56, 76, 99]

In [126]:
l

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan